In [1]:
package main

import (
    "time"
    "testing"
    "github.com/stretchr/testify/assert"
)

In [2]:
type Batch struct {
    Reference   string      `json:"reference"`  
    Sku         string      `json:"sku"`        
    Qty         int         `json:"qty"`        
    Allocations []OrderLine `json:"allocations"`
    Eta         time.Time
}

type OrderLine struct {
    ID  string `json:"id"` 
    Sku string `json:"sku"`
    Qty int    `json:"qty"`
}

type Order struct {
    Reference  string      `json:"reference"` 
    OrderLines []OrderLine `json:"orderLines"`
}

In [3]:
Batch { Sku: "TEST", }

{ TEST 0 [] 0001-01-01 00:00:00 +0000 UTC}

In [4]:
func GetDate(t time.Time) time.Time {
    return time.Date(t.Year(), t.Month(), t.Day(), 0, 0, 0, 0, time.Local)
}

In [5]:
func makeBatchAndLine(sku string, batchQty int, lineQty int) (Batch, OrderLine) {
    today := GetDate(time.Now())
    return Batch { Reference: "batch-001", Sku: sku, Qty: batchQty, Eta: today },
           OrderLine { ID: "order-123", Sku: sku, Qty: lineQty }
}

In [6]:
func (batch *Batch) Allocate(line OrderLine) {
    batch.Allocations = append(batch.Allocations, line)
}

func (batch *Batch) AvailableQty() int {
    return 0
}

In [7]:
func TestAllocatingToBatchReducesAvailableQuantity(t *testing.T) {
    today := GetDate(time.Now())
    batch := Batch { Reference: "batch-001", Sku: "SMALL-TABLE", Qty: 20, Eta: today }
    line := OrderLine { ID: "order-ref", Sku: "SMALL-TABLE", Qty: 2 }
    batch.Allocate(line)
    assert.Equal(t, 0, batch.AvailableQty())
}

In [9]:
import (
    "fmt"
    "bytes"
    "os/exec"
)

func Test() {
    cmd := exec.Command(
        "jupyter", 
        "nbconvert", 
        "01-domain-modeling-go.ipynb",
        "--to=script",
        "--TagRemovePreprocessor.remove_cell_tags",
        "hidden",
        "--output=tests/go/models_test")
    cmd.CombinedOutput()
    cmd := exec.Command("go", "test", "./tests/go")
    stdoutStderr, err := cmd.CombinedOutput()
    fmt.Printf("%s\n", stdoutStderr)
}

Test()

ok  	_/home/ykkim/notebooks/2021-msa-study/00-kick-off/tests/go	0.005s

